In [1]:
import pandas as pd
import numpy as np

In [2]:
df_clean_userID = pd.read_csv('./csv/clean_userID.csv')
df_clean_userID

,customer_id,new_id,order_product_id,browser,platform,customer_firstname,customer_lastname,customer_email,customer_gender,Majority_code,...,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18
0,0,10531,121,2,0,Julie,Staples,juliestaples40@gmail.com,2,77,...,0,0,0,0,0,0,0,0,0,0
1,0,12774,121,2,0,Alana,Locandro,alana.lannie@gmail.com,1,13,...,0,0,0,0,0,0,0,0,0,0
2,0,12285,121,7,6,Harriet,Peace,Harrietpeace@icloud.com,1,77,...,0,0,0,0,0,0,0,0,0,0
3,221,221,121,8,0,Kain,Ingham,Kaino_01@hotmail.com,0,13,...,0,0,0,0,0,0,0,0,0,0
4,0,12286,121,2,4,Sandra,Nicholl,sandinicholl@hotmail.co.uk,2,77,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5512,0,11192,202,2,4,Ali,Mohammad,fourtwenty.aggregation@gmail.com,0,38,...,0,0,0,0,0,1,0,0,0,0
5513,0,11193,202,2,4,Kirill,Kiryushin,kirill.kiryushin.o@gmail.com,0,218,...,0,0,0,0,0,1,0,0,0,0
5514,0,11194,202,2,4,Paitoon,Uthaisang,puthaisang@gmail.com,0,218,...,0,0,0,0,0,1,0,0,0,0
5515,0,11195,202,2,0,Alex,Schetinin,shatz.g@gmail.com,0,103,...,0,0,0,0,0,1,0,0,0,0


In [3]:
df_clean_userID.columns

Index(['customer_id', 'new_id', 'order_product_id', 'browser', 'platform',
       'customer_firstname', 'customer_lastname', 'customer_email',
       'customer_gender', 'Majority_code', 'order_price_paid',
       'order_payment_by', 'booked_days', 'departure_year', 'departure_month',
       'departure_day', 'departure_DayofYear', 'departure_DayofWeek',
       'sum_kids', 'sum_adults', 'private', 'group', 'family', 'cat_1',
       'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9',
       'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16',
       'cat_17', 'cat_18'],
      dtype='object')

In [4]:
def softmax(x, axis = 1):
    return np.exp(x)/np.sum(np.exp(x),axis=axis, keepdims=True)

In [5]:
use_cols = [
    'new_id', 'order_product_id', 'browser', 'platform', 'customer_gender','Majority_code', # group matrix
    'booked_days', 'order_price_paid', 
    'sum_kids', 'sum_adults', # normal and mean
    'private', 'group', 'family', # softmax
    'cat_1', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9', # softmax
    'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16','cat_17', 'cat_18' # softmax
    ]
cat_cols = ['cat_1', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9',
    'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16','cat_17', 'cat_18']

In [6]:
df_clean_userID[['order_price_paid','sum_kids', 'sum_adults']].describe().T

,count,mean,std,min,25%,50%,75%,max
order_price_paid,5517.0,3074.177814,2823.114240,0.0,1200.0,2320.0,4400.0,30020.0
sum_kids,5517.0,0.429219,0.857041,0.0,0.0,0.0,1.0,10.0
sum_adults,5517.0,2.485771,1.544137,1.0,2.0,2.0,3.0,30.0


In [7]:
df_userID = df_clean_userID[use_cols]

# constant values for replace 0 before softmax
c = 0.05
df_userID.loc[:,'private':'cat_18'] = df_userID.loc[:,'private':'cat_18'].applymap(lambda x: c if x == 0 else x)
df_userID.loc[:,'private':'cat_18']

/var/folders/5g/90_bh_2146gffg2__wzbyb780000gn/T/ipykernel_98356/202389524.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_userID.loc[:,'private':'cat_18'] = df_userID.loc[:,'private':'cat_18'].applymap(lambda x: c if x == 0 else x)


,private,group,family,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,...,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18
0,1.00,0.05,0.05,0.05,0.05,0.05,0.05,1.00,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
1,1.00,0.05,0.05,0.05,0.05,0.05,0.05,1.00,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
2,1.00,0.05,0.05,0.05,0.05,0.05,0.05,1.00,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
3,1.00,0.05,0.05,0.05,0.05,0.05,0.05,1.00,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
4,0.05,1.00,0.05,0.05,0.05,0.05,0.05,1.00,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5512,1.00,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,1.00,0.05,0.05,0.05,0.05
5513,1.00,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,1.00,0.05,0.05,0.05,0.05
5514,0.05,1.00,0.05,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,1.00,0.05,0.05,0.05,0.05
5515,0.05,0.05,1.00,0.05,0.05,0.05,0.05,0.05,0.05,0.05,...,0.05,0.05,0.05,0.05,0.05,1.00,0.05,0.05,0.05,0.05


In [8]:
df_userID['order_price_paid'] = df_userID['order_price_paid']/5000

# softmax
df_userID[cat_cols] = softmax(df_userID[cat_cols].to_numpy())
df_userID[['private', 'group', 'family']] = softmax(df_userID[['private', 'group', 'family']].to_numpy())

# normalize
df_userID[['sum_kids', 'sum_adults']] = df_userID[['sum_kids', 'sum_adults']].div(df_userID[['sum_kids', 'sum_adults']].sum(axis=1),axis=0)

/var/folders/5g/90_bh_2146gffg2__wzbyb780000gn/T/ipykernel_98356/2310491366.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_userID['order_price_paid'] = df_userID['order_price_paid']/5000
/var/folders/5g/90_bh_2146gffg2__wzbyb780000gn/T/ipykernel_98356/2310491366.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_userID[cat_cols] = softmax(df_userID[cat_cols].to_numpy())
/var/folders/5g/90_bh_2146gffg2__wzbyb780000gn/T/ipykernel_98356/2310491366.py:5: SettingWithCopyWarning: 
A value is trying to

In [9]:
df_userID[['order_price_paid','sum_kids', 'sum_adults']].describe().T

,count,mean,std,min,25%,50%,75%,max
order_price_paid,5517.0,0.614836,0.564623,0.000000,0.240000,0.464,0.880000,6.004000
sum_kids,5517.0,0.101372,0.184145,0.000000,0.000000,0.000,0.166667,0.833333
sum_adults,5517.0,0.898628,0.184145,0.166667,0.833333,1.000,1.000000,1.000000


In [10]:
df_userID

,new_id,order_product_id,browser,platform,customer_gender,Majority_code,booked_days,order_price_paid,sum_kids,sum_adults,...,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18
0,10531,121,2,0,2,77,60,0.900,0.000000,1.000000,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058
1,12774,121,2,0,1,13,53,0.900,0.000000,1.000000,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058
2,12285,121,7,6,1,77,75,0.900,0.000000,1.000000,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058
3,221,121,8,0,0,13,1,0.900,0.000000,1.000000,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058
4,12286,121,2,4,2,77,7,3.150,0.000000,1.000000,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5512,11192,202,2,4,0,38,0,0.716,0.000000,1.000000,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058,0.051058,0.051058,0.051058
5513,11193,202,2,4,0,218,1,0.796,0.000000,1.000000,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058,0.051058,0.051058,0.051058
5514,11194,202,2,4,0,218,117,1.404,0.000000,1.000000,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058,0.051058,0.051058,0.051058
5515,11195,202,2,0,0,103,0,0.796,0.333333,0.666667,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058,0.051058,0.051058,0.051058


# Matrix

In [25]:
matrix_cols = [
    # 'new_id', 'order_product_id', 'browser', 'platform', 'customer_gender','Majority_code', # group matrix
    'booked_days', 'order_price_paid', 
    'sum_kids', 'sum_adults', # normal and mean
    'private', 'group', 'family', # softmax
    'cat_1', 'cat_2', 'cat_3', 'cat_4', 'cat_5', 'cat_6', 'cat_7', 'cat_8', 'cat_9', # softmax
    'cat_10', 'cat_11', 'cat_12', 'cat_13', 'cat_14', 'cat_15', 'cat_16','cat_17', 'cat_18' # softmax
    ]

In [26]:
# user_matrix
df_user_matrix = df_userID.groupby('new_id')[matrix_cols].mean()
df_user_matrix.reset_index(inplace=True)
df_user_matrix.to_csv('./matrix/user_matrix.csv', index=False)
df_user_matrix

,new_id,booked_days,order_price_paid,sum_kids,sum_adults,private,group,family,cat_1,cat_2,...,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18
0,8,0.4,0.27836,0.000000,1.000000,0.563862,0.218069,0.218069,0.048870,0.048870,...,0.077785,0.048870,0.063849,0.048870,0.065062,0.048870,0.048870,0.048870,0.081255,0.048870
1,24,0.0,0.06400,0.000000,1.000000,0.563862,0.218069,0.218069,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058
2,130,0.0,0.45000,0.000000,1.000000,0.563862,0.218069,0.218069,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058
3,191,0.0,0.81600,0.333333,0.666667,0.218069,0.218069,0.563862,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058,0.051058,0.051058,0.051058
4,194,0.0,0.07500,0.000000,1.000000,0.563862,0.218069,0.218069,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4987,13734,0.0,0.21600,0.000000,1.000000,0.218069,0.563862,0.218069,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058
4988,13735,0.0,0.17200,0.200000,0.800000,0.218069,0.218069,0.563862,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058
4989,13736,11.0,0.14200,0.500000,0.500000,0.218069,0.218069,0.563862,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058
4990,13737,0.0,0.07200,0.000000,1.000000,0.563862,0.218069,0.218069,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058


In [27]:
# product_matrix
df_product_matrix = df_userID.groupby('order_product_id')[matrix_cols].mean()
df_product_matrix.reset_index(inplace=True)
df_product_matrix.to_csv('./matrix/product_matrix.csv', index=False)
df_product_matrix

,order_product_id,booked_days,order_price_paid,sum_kids,sum_adults,private,group,family,cat_1,cat_2,...,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18
0,29,1.714286,0.937619,0.049206,0.950794,0.497997,0.234535,0.267468,0.047233,0.047233,...,0.122132,0.047233,0.122132,0.047233,0.047233,0.047233,0.047233,0.047233,0.047233,0.047233
1,33,1.666667,1.589333,0.000000,1.000000,0.448598,0.333333,0.218069,0.113622,0.043942,...,0.043942,0.043942,0.043942,0.043942,0.043942,0.043942,0.043942,0.043942,0.043942,0.043942
2,34,7.221154,1.113040,0.116165,0.883835,0.389303,0.282905,0.327792,0.043942,0.043942,...,0.043942,0.113622,0.043942,0.043942,0.043942,0.113622,0.043942,0.043942,0.043942,0.043942
3,35,4.857143,0.764286,0.023810,0.976190,0.390966,0.366266,0.242768,0.047233,0.047233,...,0.122132,0.047233,0.122132,0.047233,0.047233,0.047233,0.047233,0.047233,0.047233,0.047233
4,38,3.250000,0.341307,0.030492,0.969508,0.497061,0.257363,0.245575,0.047233,0.122132,...,0.047233,0.047233,0.047233,0.047233,0.047233,0.047233,0.047233,0.047233,0.047233,0.047233
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,197,0.000000,0.290000,0.125000,0.875000,0.390966,0.218069,0.390966,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058,0.051058,0.051058,0.051058
99,198,2.666667,0.389667,0.023810,0.976190,0.390966,0.333333,0.275701,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058,0.051058,0.051058,0.051058
100,199,2.250000,0.801000,0.000000,1.000000,0.477414,0.304517,0.218069,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058
101,200,0.200000,0.682800,0.000000,1.000000,0.356386,0.425545,0.218069,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058


In [28]:
# browser_matrix
df_browser_matrix = df_userID.groupby('browser')[matrix_cols].mean()
df_browser_matrix.reset_index(inplace=True)
df_browser_matrix.to_csv('./matrix/browser_matrix.csv', index=False)
df_browser_matrix

,browser,booked_days,order_price_paid,sum_kids,sum_adults,private,group,family,cat_1,cat_2,...,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18
0,-1,0.000000,0.054000,0.166667,0.833333,0.390966,0.218069,0.390966,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058
1,0,0.000000,1.095533,0.362573,0.637427,0.333333,0.218069,0.448598,0.073474,0.048508,...,0.073474,0.048508,0.073474,0.048508,0.048508,0.075496,0.048508,0.048508,0.048508,0.048508
2,1,0.687500,0.266625,0.098363,0.901637,0.409490,0.285993,0.304517,0.051370,0.052568,...,0.052039,0.057106,0.052039,0.050701,0.052870,0.053492,0.050701,0.050701,0.105338,0.050701
3,2,3.067929,0.625379,0.100933,0.899067,0.408807,0.282889,0.308304,0.054622,0.052769,...,0.055931,0.072036,0.054721,0.049675,0.050367,0.064613,0.049735,0.049675,0.065958,0.051959
4,3,9.160000,0.745288,0.206667,0.793333,0.342554,0.287228,0.370218,0.055715,0.055298,...,0.055715,0.062678,0.055715,0.049724,0.049724,0.069155,0.049724,0.049724,0.055749,0.052962
5,4,4.111111,0.699493,0.074339,0.925661,0.414019,0.283385,0.302596,0.059946,0.052205,...,0.055418,0.071289,0.053095,0.049824,0.050724,0.072189,0.049824,0.049824,0.053297,0.054322
6,5,19.280000,0.888152,0.109333,0.890667,0.453208,0.231901,0.314891,0.057573,0.057155,...,0.059943,0.064326,0.057155,0.048585,0.048585,0.064326,0.048585,0.048585,0.057398,0.048585
7,6,2.466667,0.544547,0.055556,0.944444,0.494704,0.241122,0.264175,0.048140,0.062772,...,0.062772,0.057431,0.062772,0.048140,0.048140,0.068226,0.048140,0.048140,0.063581,0.048140
8,7,1.980053,0.600591,0.100604,0.899396,0.415797,0.278920,0.305284,0.054450,0.052700,...,0.057075,0.068164,0.055497,0.049977,0.051269,0.062602,0.049941,0.049941,0.068251,0.051413
9,8,2.561224,0.703629,0.110648,0.889352,0.413313,0.269820,0.316867,0.053225,0.054503,...,0.056923,0.074122,0.055462,0.049638,0.050464,0.068339,0.049638,0.049638,0.055864,0.051291


In [29]:
# platform_matrix
df_platform_matrix = df_userID.groupby('platform')[matrix_cols].mean()
df_platform_matrix.reset_index(inplace=True)
df_platform_matrix.to_csv('./matrix/platform_matrix.csv', index=False)
df_platform_matrix

,platform,booked_days,order_price_paid,sum_kids,sum_adults,private,group,family,cat_1,cat_2,...,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18
0,-1,0.000000,0.054000,0.166667,0.833333,0.390966,0.218069,0.390966,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.132020,0.051058
1,0,1.664661,0.620005,0.091206,0.908794,0.422367,0.278602,0.299031,0.054687,0.053438,...,0.055741,0.074905,0.054518,0.049787,0.050540,0.064187,0.049831,0.049787,0.059470,0.052799
2,1,9.571429,0.705914,0.238095,0.761905,0.366266,0.218069,0.415665,0.069802,0.048402,...,0.059102,0.069923,0.048402,0.048402,0.048402,0.058356,0.048402,0.048402,0.048402,0.059968
3,2,7.000000,0.870240,0.066667,0.933333,0.287228,0.425545,0.287228,0.063085,0.048105,...,0.062041,0.048105,0.062041,0.048105,0.048105,0.048105,0.048105,0.048105,0.048105,0.064297
4,3,4.181818,0.765740,0.088233,0.911767,0.409541,0.293800,0.296658,0.056927,0.053012,...,0.060066,0.066566,0.057367,0.049449,0.051122,0.062886,0.049449,0.049449,0.059159,0.049784
5,4,6.605025,0.655345,0.119266,0.880734,0.389054,0.283752,0.327194,0.053991,0.052461,...,0.056451,0.066140,0.055374,0.049431,0.049675,0.066791,0.049512,0.049431,0.075557,0.050489
6,5,2.569231,0.726020,0.094176,0.905824,0.409585,0.297867,0.292547,0.059772,0.052109,...,0.055565,0.073203,0.054493,0.049884,0.051753,0.066352,0.049884,0.049884,0.055316,0.053621
7,6,1.567158,0.570849,0.103095,0.896905,0.414788,0.277549,0.307663,0.053975,0.052662,...,0.056563,0.068314,0.055111,0.050035,0.051368,0.062491,0.050000,0.050000,0.070488,0.051368


In [30]:
# customer_gender_matrix
df_customer_gender_matrix = df_userID.groupby('customer_gender')[matrix_cols].mean()
df_customer_gender_matrix.reset_index(inplace=True)
df_customer_gender_matrix.to_csv('./matrix/customer_gender_matrix.csv', index=False)
df_customer_gender_matrix

,customer_gender,booked_days,order_price_paid,sum_kids,sum_adults,private,group,family,cat_1,cat_2,...,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18
0,0,2.383692,0.656569,0.100165,0.899835,0.413589,0.278834,0.307577,0.054862,0.052639,...,0.056822,0.071989,0.055195,0.049671,0.050771,0.066667,0.049671,0.049645,0.061587,0.051948
1,1,2.586700,0.483592,0.085735,0.914265,0.424170,0.281038,0.294792,0.053966,0.053272,...,0.055685,0.065405,0.054516,0.050029,0.050885,0.058109,0.050029,0.050029,0.079783,0.051287
2,2,4.000000,0.721190,0.140320,0.859680,0.379811,0.284532,0.335657,0.054168,0.053019,...,0.056203,0.072381,0.055534,0.049938,0.050591,0.064764,0.050047,0.049938,0.061418,0.051462


In [31]:
# CountryCode_matrix
df_customeCountryCode_matrix = df_userID.groupby('Majority_code')[matrix_cols].mean()
df_customeCountryCode_matrix.reset_index(inplace=True)
df_customeCountryCode_matrix.to_csv('./matrix/CountryCode_gender_matrix.csv', index=False)
df_customeCountryCode_matrix

,Majority_code,booked_days,order_price_paid,sum_kids,sum_adults,private,group,family,cat_1,cat_2,...,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18
0,2,1.857143,0.642394,0.087755,0.912245,0.415665,0.289423,0.294912,0.051613,0.052802,...,0.064525,0.077897,0.063336,0.049235,0.049235,0.065046,0.049235,0.049235,0.050520,0.051805
1,12,0.000000,1.214500,0.000000,1.000000,0.390966,0.390966,0.218069,0.065269,0.063964,...,0.063964,0.046544,0.046544,0.046544,0.046544,0.046544,0.046544,0.046544,0.063964,0.046544
2,13,2.803536,0.759396,0.105667,0.894333,0.415083,0.279890,0.305027,0.057244,0.053301,...,0.055843,0.076863,0.054539,0.049577,0.051327,0.069705,0.049577,0.049577,0.052537,0.051804
3,15,0.000000,1.078000,0.500000,0.500000,0.218069,0.218069,0.563862,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058
4,17,0.000000,0.260000,0.000000,1.000000,0.563862,0.218069,0.218069,0.051058,0.051058,...,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058,0.051058
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,231,20.000000,1.125000,0.226190,0.773810,0.218069,0.218069,0.563862,0.051058,0.051058,...,0.051058,0.091539,0.051058,0.051058,0.051058,0.091539,0.051058,0.051058,0.051058,0.051058
91,233,2.315018,0.718964,0.030569,0.969431,0.481531,0.271268,0.247202,0.055454,0.056367,...,0.056456,0.070981,0.055123,0.049144,0.051220,0.057635,0.049441,0.049144,0.057643,0.050034
92,241,3.444444,0.500356,0.148148,0.851852,0.333333,0.333333,0.333333,0.050267,0.050267,...,0.050267,0.084997,0.050267,0.050267,0.050267,0.093993,0.050267,0.050267,0.050267,0.050267
93,248,11.402597,0.652686,0.054978,0.945022,0.451592,0.271959,0.276450,0.056141,0.056005,...,0.057815,0.083591,0.055033,0.049332,0.050383,0.060459,0.049332,0.049332,0.051288,0.053538


In [14]:
# import sweetviz as sv
# report = sv.analyze(df_user_matrix)
# report.show_html('./report/user_matrix.html')
# df_user_matrix.to_csv('./csv/user_matrix.csv', index=False)